In [1]:
!git clone https://github.com/rajendrawarke/CarDamageDetection-SeverityPrediction.git
%cd CarDamageDetection-SeverityPrediction

Cloning into 'CarDamageDetection-SeverityPrediction'...
remote: Enumerating objects: 13699, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 13699 (delta 0), reused 0 (delta 0), pack-reused 13696 (from 2)
Receiving objects: 100% (13699/13699), 573.27 MiB | 52.71 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Updating files: 100% (15613/15613), done.
/content/CarDamageDetection-SeverityPrediction


In [2]:
!pip install -q ultralytics==8.0.210
!pip install -q roboflow
!pip install tensorflow==2.19.0
!pip install -q opencv-python-headless matplotlib seaborn scikit-learn pandas imgaug
!pip install -q scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.3/645.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.0/645.0 MB 906.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 M

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 99.9 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
import matplotlib
import seaborn as sns
import numpy as np
import ultralytics
from ultralytics import YOLO
import roboflow
import sklearn
import cv2
import os, glob, pprint
from pathlib import Path

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [4]:
print('tensorflow : ',tf.__version__)
print('matplotlib : ',matplotlib.__version__)
print('sns : ',sns.__version__)
print('np : ',np.__version__)
print('sklearn : ',sklearn.__version__)
print('ultralytics : ',ultralytics.__version__)
print('roboflow : ',roboflow.__version__)
print('cv2 : ',cv2.__version__)

pprint.pprint(sorted(glob.glob("data/**", recursive=True))[:5])

tensorflow :  2.19.0
matplotlib :  3.10.0
sns :  0.13.2
np :  2.0.2
sklearn :  1.6.1
ultralytics :  8.0.210
roboflow :  1.2.11
cv2 :  4.10.0
[]


In [5]:
## As data.yaml file is already defined in the roboflow dataset, we will not execute below code to create our own yaml file for training the yolo model
# data_yaml = """
# train: dataset/train/images
# val: dataset/valid/images
# nc: 3
# names: ['scratch','dent','glass_shatter']
# """
# with open("data.yaml","w") as f:
#     f.write(data_yaml.strip())
# print("Wrote data.yaml")

In [6]:
import torch
def unsafe_torch_load(*args, **kwargs):
    kwargs['weights_only'] = False
    return torch.serialization.orig_load(*args, **kwargs)

if not hasattr(torch.serialization, 'orig_load'):
    torch.serialization.orig_load = torch.load
    torch.load = unsafe_torch_load

In [7]:
model = YOLO("yolov8n.pt")
model.train(data="datasets/data.yaml", epochs=15, imgsz=640, batch=8, name="car_damage_yolov8n")

100%|██████████| 6.23M/6.23M [00:00<00:00, 105MB/s]
New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.210 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (AMD EPYC 7B13)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=datasets/data.yaml, epochs=15, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=car_damage_yolov8n, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x783055610cb0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004

In [8]:
# api keyfor wandb login : f08db23a1f25a5634bb41c3cf875470469d26c46

In [10]:
detector_path = "runs/detect/car_damage_yolov8n/weights/best.pt"
detector = YOLO(detector_path)

In [13]:
os.makedirs("crops", exist_ok=True)
val_images = sorted(list(Path("datasets/valid/images").glob("*.*")))
crop_count = 0

In [14]:
for img_path in val_images:
    res = detector.predict(source=str(img_path), imgsz=640, conf=0.25, verbose=False)
    r = res[0]
    img = cv2.imread(str(img_path))
    h,w = img.shape[:2]
    for i,box in enumerate(r.boxes):
        xyxy = box.xyxy[0].cpu().numpy()  # [x1,y1,x2,y2]
        conf = float(box.conf[0].cpu().numpy())
        cls = int(box.cls[0].cpu().numpy())
        x1,y1,x2,y2 = map(int, xyxy)
        pad = 8
        x1, y1 = max(0,x1-pad), max(0,y1-pad)
        x2, y2 = min(w-1,x2+pad), min(h-1,y2+pad)
        crop = img[y1:y2, x1:x2]
        fname = f"crops/{img_path.stem}_box{i}_cls{cls}_conf{int(conf*100)}.jpg"
        cv2.imwrite(fname, crop)
        crop_count += 1

print(f"Saved {crop_count} crops to crops/")

Saved 2001 crops to crops/


In [ ]:
# Create severity dataset
# filename,severity
# crops/img1_box0_cls0_conf98.jpg,minor
# crops/img2_box1_cls2_conf76.jpg,moderate

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

labels_csv = "severity_labels.csv"
assert os.path.exists(labels_csv), "Create severity_labels.csv with columns (filename,severity)"

labels_df = pd.read_csv(labels_csv)
labels_df['severity'] = labels_df['severity'].str.lower().str.strip()

label_map = {'minor':0, 'moderate':1, 'severe':2}
labels_df['label'] = labels_df['severity'].map(label_map)
labels_df = labels_df[labels_df['label'].notnull()]

train_df, val_df = train_test_split(labels_df, test_size=0.2, stratify=labels_df['label'], random_state=42)

IMG_SIZE = (224,224)
BATCH = 32
AUTOTUNE = tf.data.AUTOTUNE

def process_path(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image, label

def df_to_dataset(df, shuffle=True, batch_size=BATCH):
    paths = df['filename'].astype(str).values
    labels = df['label'].astype(int).values
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(lambda x,y: tf.py_function(lambda a,b: process_path(a,b), [x,y], Tout=(tf.float32, tf.int32)),
                num_parallel_calls=AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(1024)
    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds

train_ds = df_to_dataset(train_df, shuffle=True)
val_ds = df_to_dataset(val_df, shuffle=False)

print("Train size:", len(train_df), "Val size:", len(val_df))

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

num_classes = 3

base = EfficientNetB0(include_top=False, input_shape=(224,224,3), weights='imagenet')
base.trainable = False

inputs = layers.Input(shape=(224,224,3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

callbacks = [
    ModelCheckpoint("best_severity_model.h5", monitor='val_accuracy', save_best_only=True, mode='max'),
    EarlyStopping(monitor='val_accuracy', patience=6, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
]

history = model.fit(train_ds, validation_data=val_ds, epochs=15, callbacks=callbacks)

In [ ]:
val_loss, val_acc = model.evaluate(val_ds)
print("Val loss:", val_loss, "Val acc:", val_acc)

y_true = []
y_pred = []
for imgs, labels in val_ds:
    preds = model.predict(imgs)
    preds = np.argmax(preds, axis=1)
    y_pred.extend(preds.tolist())
    y_true.extend(labels.numpy().tolist())

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred, target_names=['minor','moderate','severe']))
print(confusion_matrix(y_true, y_pred))

In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont

from ultralytics import YOLO
detector = YOLO(detector_path)
severity_model = tf.keras.models.load_model("best_severity_model.h5")

def infer_and_visualize(img_path, conf_thresh=0.25):
    img = cv2.imread(img_path)
    h,w = img.shape[:2]
    res = detector.predict(source=str(img_path), imgsz=640, conf=conf_thresh, verbose=False)
    r = res[0]
    pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil)
    for box in r.boxes:
        xyxy = box.xyxy[0].cpu().numpy().astype(int)
        conf = float(box.conf[0].cpu().numpy())
        cls = int(box.cls[0].cpu().numpy())
        x1,y1,x2,y2 = xyxy
        pad = 8
        x1, y1 = max(0,x1-pad), max(0,y1-pad)
        x2, y2 = min(w-1,x2+pad), min(h-1,y2+pad)
        crop = img[y1:y2, x1:x2]
        crop_resized = cv2.resize(crop, (224,224))/255.0
        pred = severity_model.predict(np.expand_dims(crop_resized, 0))
        severity = ['minor','moderate','severe'][int(np.argmax(pred))]
        draw.rectangle([x1,y1,x2,y2], outline="red", width=3)
        draw.text((x1, max(0,y1-18)), f"{severity} {conf:.2f}", fill="yellow")
    display(pil)

infer_and_visualize("dataset/valid/images/example.jpg", conf_thresh=0.3)